In [12]:
!pip install pymorphy2
import nltk
nltk.download('stopwords')
import re
import json
import os
import numpy as np
import pymorphy2
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from math import log
import collections
import sklearn
from collections import Counter

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
# stop_words.extend()

morph = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()


from ner.corpus import Corpus
import json
from ner.utils import md5_hashsum, download_untar
from glob import glob
from ner.network import NER


# Check existance of the model by hashsum
if md5_hashsum(sorted(glob('model/*'))) != 'fd50a27b96b24cdabdda13795a3baae7':
    # Download and extract model
    download_url = 'http://lnsigo.mipt.ru/export/models/ner/ner_model_total_rus.tar.gz'
    download_path = 'model/'
    download_untar(download_url, download_path)

# Load network params
with open('model/params.json') as f:
    network_params = json.load(f)


c_n = Corpus(dicts_filepath='model/dict.txt')
network =  NER(c_n, verbouse=False, pretrained_model_filepath='model/ner_model', **network_params)
from ner.utils import tokenize, lemmatize

!pip install natasha
from natasha import (NewsNERTagger,  NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, Segmenter, Doc)
import pandas as pd
# !pip3 install git+https://github.com/deepmipt/ner
import ner

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
answers = pd.read_csv('answers_base.csv', encoding = 'windows-1251', sep = ';')
queries = pd.read_csv('queries_base.csv', encoding = 'windows-1251', sep = ';')

ans = answers[['Номер связки','Текст вопросов']].dropna(axis = 0, how ='any')
qw = queries[['Текст вопроса', 'Номер связки\n']].dropna(axis = 0, how ='any')
qw.rename(columns={'Текст вопроса': 'Текст вопросов', 'Номер связки\n': 'Номер связки'}, inplace=True)
train = pd.concat([ans, qw.iloc[0:int(qw.shape[0]*0.7), :]]) #train

queries2 = qw.iloc[int(qw.shape[0]*0.7):, 0].tolist()
test = dict(zip(qw.iloc[int(qw.shape[0]*0.7):, 0], qw.iloc[int(qw.shape[0]*0.7):, 1])) # test

In [32]:
def tokenize_ru(sentence):
    sentence = sentence.replace('\n', ' ').replace('/', ' ')
    sentence = re.sub(r'[\'"”\,\!\?\.\-\(\)\[\]\:\;\»\«\>\—]', ' ', str(sentence).rstrip("']"))
    sentence = re.sub(r'[0-9]', ' ', str(sentence))
    sentence = sentence.lower()
    tokens = sentence.split()
    tokens = [i for i in tokens if (i not in stop_words)]
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    tokens = ' '.join(tokens)
    return tokens

In [13]:
!pip install natasha
from natasha import (NewsNERTagger,  NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, Segmenter, Doc)
import pandas as pd


def preprocess_with_natasha(text: str) -> str:
    doc = Doc(text)
    segmenter = Segmenter()
    emb = NewsEmbedding()

    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    list_word_ner = [word.text for word in doc.spans]
    print(list_word_ner)
    for word in list_word_ner:
        text = text.replace(word, '')
    return text

def print_predict(sentence, network):
    tokens = tokenize(sentence)
    tokens_lemmas = lemmatize(tokens)
    list_words_ner = []
    tags = network.predict_for_token_batch([tokens_lemmas])[0]
    for token, tag in zip(tokens, tags):
        if tag != 'O':
            list_words_ner.append(token)
    return list_words_ner

def preprocess_with_deepmipt(text: str) -> str:
    list_words_ner = print_predict(text, network)
    for word in list_words_ner:
        text = text.replace(word, '')
    return text

     |████████████████████████████████| 34.4MB 83kB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26100 sha256=648cd0077ed2a4b62afe49fcabe8dcee195ecccb3278671300c3a995024a4e2e
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
  Cloning https://github.com/deepmipt/ner to /tmp/pip-req-build-d0o510ki
  Running command git clone -q https://github.com/deepmipt/ner /tmp/pip-req-build-d0o510ki
     |████████████████████████████████| 17.0MB 200kB/s 
     |████████████████████████████████| 43.6MB 70kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 7.1MB 8.7MB/s 
     |█████

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

100%|██████████| 44.3M/44.3M [00:09<00:00, 4.82MB/s]


Extracting /usr/local/lib/python3.6/dist-packages/ner/extractor/../model/ner_model_total_rus.tar.gz archive into /usr/local/lib/python3.6/dist-packages/ner/extractor/../model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /usr/local/lib/python3.6/dist-packages/ner/extractor/../model/ner_model


In [33]:
def create_corpus(train_text, NER = False):
  corpus = []
  for question in train_text['Текст вопросов']:
      question = question.replace('\n', ' ').replace('/', ' ')
      if NER == False:
        pass
      else:
        question = preprocess_with_deepmipt(question)
      words_doc = tokenize_ru(question)
      corpus.append(words_doc)
  return corpus

In [29]:
def create_mat_tfidf(corpus):
  X = vectorizer.fit_transform(corpus)
  X.toarray()
  return X

def bm25(tf_q_d, l, N, nq, corpus) -> float:
    k = 2.0
    b = 0.75
    aver = Average(corpus)
    TF = (tf_q_d * (k+1))/(tf_q_d + k*(1 - b + b*(l/aver)))
    IDF = log((N-nq+0.5)/(nq + 0.5))
    result = IDF*TF
    return result

def Average(lst):
    lst2 = []
    d = {}
    for doc in lst:
        lst2.append(len(doc.split(' ')))
    return sum(lst2) / len(lst)

def create_mat_bm25(corpus):
  N = len(corpus)
  nq = Counter(full_data)
  matr = np.zeros((N, len(nq)))

  for i, doc in enumerate(corpus):
      doc = doc.split(' ')
      tf_q_d = Counter(doc)
      l = len(doc)
      for j, word in enumerate(set(full_data)):
          if tf_q_d[word] == 0:
              matr[i, j] = 0
          else:
              matr[i, j] = bm25(tf_q_d[word], l, N, nq[word], corpus)
  return matr


In [24]:
# перевод запросов в соответствующий вид


def to_tiidf(query):
    query = query.replace('\n', ' ').replace('/', ' ')
    query = tokenize_ru(query)
    # print(query)
    vect = vectorizer.transform([query]).toarray()
    return vect 

def query_f_tfidf(text, matrix):
    # ner_text = preprocess_with_natasha(text)
    new_doc = to_tiidf(text).T
    proximity = []
    for row in matrix:
        proximity.append(float(row.dot(new_doc)))
    number_sv = train['Номер связки'].tolist()
    dict_ans = dict(zip(proximity, number_sv))
    y_pred = dict_ans[sorted(dict_ans.keys(), reverse=True)[0]]
    y_true = test[text]
    return y_pred, y_true





def to_bm25(query, full_data):
    vect = np.zeros((1, len(full_data)))
    query = query.replace('\n', ' ').replace('/', ' ')
    query = tokenize_ru(query)
    for word in query.split(' '):
        if word in full_data:
            vect[0, full_data.index(word)] = 1
    return vect

def query_f_bm25(text, matrix):
    # ner_text = preprocess_with_natasha(text)
    new_doc = to_bm25(text, list(set(full_data))).T
    proximity = []
    for row in matrix:
        r = row.reshape(1, row.shape[0])
        proximity.append(float(r.dot(new_doc)))
    number_sv = train['Номер связки'].tolist()
    dict_ans = dict(zip(proximity, number_sv))

    y_pred = dict_ans[sorted(dict_ans.keys(), reverse=True)[0]]
    y_true = test[text]
    return y_pred, y_true

In [18]:
def len_acc(fun, matr):
  y_pred = []
  y_true = []
  for text in test.keys():
      p, t = fun(text, matr)
      y_true.append(t) 
      y_pred.append(p)
  return sklearn.metrics.accuracy_score(y_true, y_pred)

In [127]:
corpus = create_corpus(train) # создаю корпус для текстов с неудаленными NER

это accuracy для tf-idf c ner

In [129]:
X = create_mat_tfidf(corpus)
full_data = ' '.join(corpus).split(' ')
len_acc(query_f_tfidf, X)

0.35798816568047337

это accuracy для bm25 c ner

In [131]:
X1 = create_mat_bm25(corpus)
full_data = ' '.join(corpus).split(' ')
len_acc(query_f_bm25, X1)

0.5207100591715976

это accuracy для tf-idf без ner

In [16]:
corpus_not_NER = create_corpus(train, True) # создаю корпус для текстов с удаленными NER

In [25]:
# X_not_NER = create_mat_tfidf(corpus_not_NER)
# full_data = ' '.join(corpus_not_NER).split(' ')
len_acc(query_f_tfidf, X_not_NER)

0.2869822485207101

это accuracy для bm25 без ner

In [30]:
X_not_NER1 = create_mat_bm25(corpus_not_NER)
full_data = ' '.join(corpus_not_NER).split(' ')
len_acc(query_f_bm25, X_not_NER1)

0.4985207100591716